In [1]:
import pandas as pd, MySQLdb
import datetime

In [2]:
conn = MySQLdb.connect(host='localhost',user='root',passwd='',db='qyw', charset='utf8')

In [3]:
df_new_user = pd.read_sql('''
    SELECT * FROM qyw_4th_event_new_user;
    ''',
    con=conn)
df_old_user = pd.read_sql('''
    SELECT * FROM qyw_4th_event_old_user;
    ''',
    con=conn)

In [4]:
import datetime
df_new_user['VISIT_TIME'] = df_new_user['VISIT_TIME'].apply(lambda x: datetime.date(x.year,x.month,x.day))
df_old_user['VISIT_TIME'] = df_old_user['VISIT_TIME'].apply(lambda x: datetime.date(x.year,x.month,x.day))

In [5]:
df_new_user_ops = pd.DataFrame(df_new_user.groupby(['VISIT_TIME']).CASE_ID.agg(lambda x: len(x)))
df_new_user_cnt = pd.DataFrame( df_new_user.groupby(['VISIT_TIME']).USER_ID.nunique())
df_old_user_ops = pd.DataFrame(df_old_user.groupby(['VISIT_TIME']).CASE_ID.agg(lambda x: len(x)))
df_old_user_cnt = pd.DataFrame( df_old_user.groupby(['VISIT_TIME']).USER_ID.nunique())

In [6]:
df_new_user_ops = df_new_user_ops.rename(columns={"CASE_ID": "新用户操作记录数"})
df_old_user_ops = df_old_user_ops.rename(columns={"CASE_ID": "老用户操作记录数"})
df_new_user_cnt = df_new_user_cnt.rename(columns={"USER_ID": "新用户在线用户数"})
df_old_user_cnt = df_old_user_cnt.rename(columns={"USER_ID": "老用户在线用户数"})

In [7]:
df_new_user_ops['VISIT_TIME']=df_new_user_ops.index;
df_old_user_ops['VISIT_TIME']=df_old_user_ops.index;
df_new_user_cnt['VISIT_TIME']=df_new_user_cnt.index;
df_old_user_cnt['VISIT_TIME']=df_old_user_cnt.index;
df_user = pd.merge(df_new_user_ops, df_old_user_ops, on='VISIT_TIME')
df_user = pd.merge(df_user, df_new_user_cnt, on='VISIT_TIME')
df_user = pd.merge(df_user, df_old_user_cnt, on='VISIT_TIME')
df_user.index = df_user['VISIT_TIME']
del df_user['VISIT_TIME']
# df_user = df_user.transpose()
# df_user.columns
# df_user.index
df_user

,新用户操作记录数,老用户操作记录数,新用户在线用户数,老用户在线用户数
VISIT_TIME,,,,
2016-03-08,8718,24033,198,676
2016-03-09,10252,26304,226,739
2016-03-10,8669,24595,189,696
2016-03-11,7637,25754,200,765
2016-03-12,7781,19411,174,586
2016-03-13,6412,17475,141,538
2016-03-14,9253,37106,198,1116
2016-03-15,9390,33846,210,898


In [14]:
df_user_new_cat = pd.read_sql('''
    SELECT CATEGORY, COUNT(DISTINCT USER_ID) FROM (SELECT * FROM qyw_4th_event_new_user ORDER BY VISIT_MEAN) AS t1 INNER JOIN (SELECT * FROM qyw_4th_business_dict ORDER BY MEAN) AS t2 ON t1.VISIT_MEAN = t2.MEAN GROUP BY CATEGORY;
    ''',
    con=conn)
df_user_old_cat = pd.read_sql('''
    SELECT CATEGORY, COUNT(DISTINCT USER_ID) FROM (SELECT * FROM qyw_4th_event_old_user ORDER BY VISIT_MEAN) AS t1 INNER JOIN (SELECT * FROM qyw_4th_business_dict ORDER BY MEAN) AS t2 ON t1.VISIT_MEAN = t2.MEAN GROUP BY CATEGORY;
    ''',
    con=conn)

In [15]:
df_user_new_cat = df_user_new_cat.rename(columns={"COUNT(DISTINCT USER_ID)": "新用户数", "CATEGORY":"业务类型"})
df_user_old_cat = df_user_old_cat.rename(columns={"COUNT(DISTINCT USER_ID)": "老用户数", "CATEGORY":"业务类型"})

In [ ]:
# df_new_user_ops = df_new_user_ops.transpose()
# df_old_user_ops = df_old_user_ops.transpose()
# df_new_user_cnt = df_new_user_cnt.transpose()
# df_old_user_cnt = df_old_user_cnt.transpose()

In [ ]:
# print df_new_user_ops
# print df_old_user_ops
# print df_new_user_cnt
# print df_old_user_cnt

In [18]:
df_user_cat = pd.merge(df_user_new_cat, df_user_old_cat, on='业务类型', how='outer')

In [20]:
df_user_cat.fillna(0)

,业务类型,新用户数,老用户数
0,公共,1536,3616
1,医院参数,1468,3496
2,就医指南,48,247
3,我的报告单,1,6
4,我的账单,1,3
5,我的退款,8,49
6,消息,729,1129
7,系统架构,9,77
8,门诊业务,74,242
9,预约挂号,1536,3619
